# Algal Bloom Detection
## or
# Earth Observation and Data Science: data preparation

### Example how to:

* Reproject spatial data;
* Clean scenes from the clouds;
* Detect harmful algal blooms in the Landsat 8 scenes by band indices taken from the literature review.

In [ ]:
# Import these modules and functions: os, numpy, pandas, rasterio, fiona
# from_epsg from fiona.crs, Proj from pyproj, transform from pyproj,
# pyplot from matplotlib, clip_area from scripts.clip_area,
# read_landsat_images from scripts.read_landsat_images, show_band from scripts.show_band
# set %matplotlib inline or %matplotlib notebook
# optionally set np.seterr(divide='ignore', invalid='ignore')

# Base
import os
import numpy as np
import pandas as pd

# Spatial analysis
import rasterio as rio
import fiona as fio
from fiona.crs import from_epsg
from pyproj import Proj, transform

# Visualization
import matplotlib.pyplot as plt

# Additional scripts
from scripts.clip_area import clip_area
from scripts.read_landsat_images import read_landsat_images
from scripts.show_band import show_band

%matplotlib notebook

# Runetime Warnings Disabling
np.seterr(divide='ignore', invalid='ignore')

In [ ]:
# Class for preparation of Landsat bands is given for you. If you would like to implement it
# by yourself then remember to preapre data structure which contains paths to the parent
# folders with Landsat 8 bands and has field dedicated for the timestamp


class RasterTimeSeries:
    """Class prepares dataframe object with addresses to the folders with Landsat bands. Dataframe index is a time when each scene was retrieved.
    It is initialized with the folder which contains all landsat bands.
    
    Class methods:
    prepare_directories(main_folder=None) - method prepares a list of folders with satellite images. Method performs search in the parent directory
    given by the user or if it is None then it is taken from the class itself."""
    
    def __init__(self, main_folder):
        self.main_folder = main_folder
        self.landsat_folders = None
        self.landsat_dataframe = None
    
    def prepare_directories(self, main_folder=None):
        """Function prepares list with Landsat images directories in the given folder.
        input 1: main_folder (string)
        If None then self.main_folder is used as the parent directory of the folders with satellite images.
        output: prepared_list - list with all folders with satellite images."""
        
        if main_folder:
            folder_name = os.listdir(main_folder)
        else:
            folder_name = self.main_folder
        raw_list = os.listdir(folder_name)
        prepared_list = []
        for folder in raw_list:
            if ((folder.startswith('LC')) and not (folder.endswith('.tar.gz'))):
                prepared_list.append(folder_name + '/' + folder + '/')
        self.landsat_folders = prepared_list
        return prepared_list
    
    @staticmethod
    def read_timestamp(folder):
        """Function reads date of acquisition of each Landsat 8 scene based on the folder with MTL file.
        input 1: folder - string with Landsat folder (MTL file must be placed in it).
        output: string with the scene acquisition time"""
    
        files = os.listdir(folder)
        for file in files:
            if file.startswith('LC') and file.endswith('MTL.txt'):
                address = folder + '/' + file
                with open(address, 'r') as f:
                    for line in f:
                        line = ''.join(line.split())
                        if line.startswith('DATE_ACQUIRED='):
                            line = line.replace('DATE_ACQUIRED=', '')
                            return line
    
    def prepare_dataframe(self):
        """Function prepares dataframe with index column as a date of acquisition and the main column DIR which points
        to the directories with Landsat 8 images. Function takes one argument: the list of folders with Landsat 8 datasets.
        output: pandas dataframe with indexes set to the dates of acquisition and column indices folders with Landsat images."""
        
        # DataFrame preparation
        time_series = pd.DataFrame(self.landsat_folders, columns=['DIR'])
        time_series.index = time_series['DIR'].apply(lambda x: str(pd.to_datetime(self.read_timestamp(x)).date()))
        time_series.index.name = 'DATE'
        time_series = time_series.sort_index()
        self.landsat_dataframe = time_series
        return time_series
    
    def __str__(self):
        if self.landsat_folders is None:
            return('Parent folder of all scenes: {}.\
            \nList of subfolders is not created.\
            \nDataframe with dates is not created.'.format(
            self.main_folder))
        elif self.landsat_dataframe is None:
            return('Parent folder of all scenes: {}\
            \nList of subfolders has {} directories.\
            \nDataframe with dates is not created.'.format(
            self.main_folder, len(self.landsat_folders)))
        else:
            return('Parent folder of all scenes: {}\
            \nList of subfolders has {} directories.\
            \nDataframe is created. List of all dates: {}.'.format(
            self.main_folder,
            len(self.landsat_folders),
            self.landsat_dataframe.index.values))

In [ ]:
# Create RasterTimeSeries instance with directory which contains Landsat 8 folders

folder = 'data/rasters'
algal_blooms = RasterTimeSeries(folder)
print(algal_blooms)

In [ ]:
# Prepare list of Landsat 8 directories and timestamp of each scene (it could be read from the
# directory name). Use prepare_directories() and then prepare_dataframe() methods of 
# the RasterTimeSeries class.

algal_blooms.prepare_directories()
print(algal_blooms)

In [ ]:
algal_blooms.prepare_dataframe()
print(algal_blooms)

In [ ]:
# Again, class for Vector Files reading and reprojecting is given here. It is hard to prepare
# it if you've never worked with the polygon files.

# The main class for vector file handling

class VectorData:
    """Class reads, stores, shows and reprojects vector shapefiles.
    It is initialized with filename.
    
    Class methods:
    get_vector_data(parameters='all', write_params=True) - method reads vector data and parse its parameters such as geometry,
    properties, crs and schema into dictionaries. Method may store parameters inside the object if write_params is set
    to True. Method returns desired parameters.
    reproject_geometry(destination_epsg, update=False) - method changes projection of each point inside the Polygon into
    projection given as an epsg number. It may update object geometry if the update parameter is set to True. Method returns
    reprojected geometry dictionary.
    show_vector_data(geometry=None) - method shows polygon. If geometry is not given then method uses class geometry."""
    
    def __init__(self, filename):
        self.v_file = filename
        self.geometry = {}
        self.properties = None
        self.vec_crs = None
        self.vec_schema = None
        
    def get_vector_data(self, parameters='all', write_params=True):
        """
        Function reads vector and its parameters and return them and / or store them inside the objects instance.
        input 1: parameters (string)
        parameters: 'all', 'none', 'geometry', 'properties', 'crs', 'schema'
        'all': returns tuple with geometry, properties, crs and schema;
        'none': does not return anything;
        'geometry', 'properties', 'crs' or 'schema': returns chosen parameter.
        input 2: write_params (bool)
        write_params: True, False
        True: store all parameters in the object
        False: do not store anything in the object
        output: chosen parameters as a dict"""
        
        with fio.open(self.v_file, 'r') as masking_region:
            geometry = [feature["geometry"] for feature in masking_region]
            properties = [feature['properties'] for feature in masking_region]
            vec_crs = masking_region.crs
            vec_schema = masking_region.schema
        
        if write_params:
            print('--- Object geometry, properties, crs and schema updated ---')
            self.geometry = geometry
            self.properties = properties
            self.vec_crs = vec_crs
            self.vec_schema = vec_schema
        else:
            print('--- Object parameters not updated ---')
            
        output_dict = {'all': ({'geometry': geometry, 'properties': properties, 'crs': vec_crs, 'scheme': vec_schema}),
                      'none': 0,
                      'geometry': geometry,
                      'properties': properties,
                      'crs': vec_crs,
                      'schema': vec_schema}
        try:
            return output_dict[parameters]
        except KeyError:
            raise KeyError("Parameter not available. Available parameters: 'all', 'none', 'geometry', 'properties', 'crs', 'schema'")
      
    def reproject_geometry(self, destination_epsg, update=False):
        """
        Function reprojects vector geometry and updates it.
        input 1: destination_epsg (string or int)
        input 2: update (bool)
        True: update object's geometry
        False: do not update object's geometry
        output: reprojected geometry as a dict"""

        try:
            destination_crs = from_epsg(destination_epsg)
            proj_crs_in = Proj(self.vec_crs)
            coordinates_list = self.geometry[0]['coordinates']
        except RuntimeError:
            raise ValueError('Given EPSG is wrong or it is not stored in fiona to run from_epsg method')
        except KeyError:
            error_text = 'Please, update your object parameters ' \
                         'by get_vector_data method with write_params set to True.'
            raise ValueError(error_text)
        proj_crs_out = Proj(init=destination_crs['init'])
        projected_geometries = []

        for geometries in coordinates_list:
            projected_g = []

            # Check if it is a multipart geometry
            if len(geometries) == 1:
                for g in geometries[0]:
                    transformed = transform(proj_crs_in, proj_crs_out, g[0], g[1])
                    points = (transformed[0], transformed[1],)
                    projected_g.append(points)
            else:
                for g in geometries:
                    transformed = transform(proj_crs_in, proj_crs_out, g[0], g[1])
                    points = (transformed[0], transformed[1],)
                    projected_g.append(points)
            projected_geometries.append(projected_g)
        geometry_dict = {'coordinates': projected_geometries, 'type': 'Polygon'}

        if update:
            self.geometry = [geometry_dict]
            self.vec_crs = destination_crs

        return [geometry_dict]
    
    def show_vector_data(self, geometry=None):
        """
        Function shows vector geometry.
        input 1: geometry (dict, geometry retrieved as a parameter from the vector data type).
        If geometry is not given then method takes it from the objects instance."""
        
        if geometry:
            g = geometry
        else:
            g = self.geometry
            
        try:
            coordinates_list = g[0]['coordinates'][0]
        except KeyError:
            raise ValueError(
                'Geometry is not defined. Please, update geometry with get_vector_data method or provide valid geometry object to the method.')
            
        coordinates_array = np.asarray(coordinates_list)
        plt.figure()
        plt.plot(coordinates_array[:, 0], coordinates_array[:, 1])
        plt.show()

In [ ]:
# You may test this class and its methods with vector file of Poland. It is stored in
# data/vector/Polska.shp file

# First read file by the class constructor and get all parameters of this file 
# by get_vector_data() method and then show it by show_vector_data() method.

poland = 'data/vector/Polska.shp'
vector_poland = VectorData(poland)
vector_poland.get_vector_data()
vector_poland.show_vector_data()

In [ ]:
# Read CRS of this data. (VectorData class has vec_crs object, use it)

vector_poland.vec_crs

In [ ]:
# Change crs and projection to the epsg:32634 and save it to the new variable
# Use reproject_geometry method. Do not overwrite actual CRS and set update
# parameter to False.
# Show the new geometry. Compare axes of this plot and plot in original CRS.

epsg = 32634
changed_crs = vector_poland.reproject_geometry(epsg, update=False)
vector_poland.show_vector_data(changed_crs)

In [ ]:
# Additionally: plot both projections on the same figure to see that they cover different
# geolocations. Use show_multiple_vectors function and pass into it list of your vectors.

def show_multiple_vectors(geometries):
    """
    Function shows vector geometry.
    input 1: list of geometries 
    (dictionaries, geometry retrieved as a parameter from the vector data type)."""

    plt.figure()
    for geometry in geometries:
        try:
            coordinates_list = np.asarray(geometry[0]['coordinates'][0])
            plt.scatter(coordinates_list[:, 0], coordinates_list[:, 1], s=0.1)
        except KeyError:
            raise ValueError(
                'Geometry is not defined. Please, update geometry with get_vector_data method or provide valid geometry object to the method.')
    plt.show()

show_multiple_vectors([changed_crs, vector_poland.geometry])

In [ ]:
# Prepare new function to get the epsg number from Landsat raster. Use rasterio.open method.
# You shouldn't read whole raster, if you use rasterio.open to get the Landsat file
# then you may read only its crs. You will get dict CRS({'init': 'epsg:32634'})
# EPSG is a number after epsg: string (32634) and this should be returned by the function

def get_epsg_from_raster(raster_address):
    """Function reads raster data and gets its coordinate reference system.
    Then it is converted to the European Petroleum Survey Group (EPSG) reference number.
    Function was tested only with the Landsat rasters."""
    with rio.open(raster_address) as f:
        band_crs = f.crs
        destination_epsg = band_crs['init'][5:]
        destination_epsg = int(destination_epsg)
    return destination_epsg

In [ ]:
landsat_images = read_landsat_images(algal_blooms.landsat_dataframe['DIR'][0])
epsg = get_epsg_from_raster(landsat_images[1])

In [ ]:
# Now you have the epsg number and you know how to reproject vector geometry.
# Now is the time to prepare area of interest for the further study. Read file 
# zatoka_gdanska.shp and prepare its geometry.
# Do not reproject this file yet. Try to use function clip_area:
# clip_area(vector_data.geometry, address_to_landsat_image, output_filename.tif)
# Read the error returned by rasterio. It is important to understand these errors.

filename = 'data/vector/zatoka_gdanska.shp'
gdansk_bay = VectorData(filename)
gdansk_bay.get_vector_data()
clip_area(gdansk_bay.geometry, landsat_images[1], 'data/failed_test.tif')

In [ ]:
# Reproject vector geometry and try to clip raster area to the new file with updated geometry
# You may overwrite data (vector file)
# Save clipped file as a "test.tif"
# Show "test.tif". Read file with rasterio.open and show band with show_band() function.

gdansk_bay.reproject_geometry(epsg, update=True)
clip_area(gdansk_bay.geometry, landsat_images[5], 'data/test.tif')
with rio.open('data/test.tif', 'r') as test_file:
    band = test_file.read(1)
    show_band(band)

In [ ]:
# From this point you know how to prepare single band for analysis.
# The next step is to make process automatic so you cannot write functions for each raster.
# Maybe you've noticed that Landsat has many bands - we will use only few of them so your
# method should include this fact.
# Below is an example of implementation. Explanations for this implementation are presented
# in the article.

def clip_all(dataset, clipping_vector, bands=(2, 3, 4), folder_name='clipped/'):
    """Function performs clipping of all Landsat 8 bands given in the DIR column of the df parameter.
    input 1: dataset - dataframe with DIR column where each record points to the directory with a Landsat bands
    input 2: clipping_vector - vector geometry which is used as a area of interest
    input 3: bands=(2, 3, 4) - tuple of bands which should be clipped. Bands 2, 3, 4 are used for the BGR image creation.
    input 4: folder_name='clipped/' - directory where all clipped bands should be stored.
    
    output: [correct, damaged] - list of dicts with keys (dates) and values - correct images
    and damaged images. Damaged images were not process and they not exist in the destination folder."""
    
    data_range = len(dataset)
    damaged = {}
    correct = {}
    for i in range(0, data_range):
        date_str = dataset.index[i]
        images = read_landsat_images(dataset['DIR'][i])
        for band_number in bands:
            band_to_clip = images[band_number]
            filename = folder_name + 'LC_band' + str(band_number) + '_' + date_str + '.tif'
            try:
                clip_area(clipping_vector, band_to_clip, filename)
            except (rio.errors.WindowError, ValueError):
                print('\nPolygon does not cover raster area. Error occurs when projections are different or raster extent is translated and it does not overlap polygon extent.\n')
                try: 
                    damaged[date_str].append(filename)
                except KeyError:
                    damaged[date_str] = [filename]
                pass
            
            # Check file
            try: 
                damaged[date_str]
                print('File: {} not saved. Raster has not been clipped.'.format(
                filename))
            except KeyError:
                try:
                    with rio.open(filename) as tf:
                         pass
                except rio.RasterioIOError:
                    raise IOError('File was: {} not saved successfully'.format(
                    filename))
                else:
                    print('File: {} saved successfully'.format(
                    filename))
                    try: 
                        correct[date_str].append(filename)
                    except KeyError:
                        correct[date_str] = [filename]
    return [correct, damaged]

In [ ]:
# Prepare tuple of bands (1, 2, 3, 4, 5) and clip multiple files with this tuple.

my_bands = (1, 2, 3, 4, 5)  # bands for EVI, Kab1 and Kab2 calculation
my_folder_name = 'data/clipped/'
prepared_files = clip_all(algal_blooms.landsat_dataframe,
                          gdansk_bay.geometry, 
                          my_bands,
                          my_folder_name)

In [ ]:
# Prepare pandas DataFrame from generated dict - take into account only correct values.
# Use method pandas.DataFrame.from_dict(). Set orient parameter to 'index'. Show head() of this
# DataFrame.

geo_df = pd.DataFrame.from_dict(prepared_files[0], orient='index')
geo_df.head()

In [ ]:
# Try to show band from the date '2018-08-03'. Use DataFrame generated in the previous step.
# Use df.loc[row_indexer, column_indexer] to get the proper value
# There's a little problem in there...

with rio.open(geo_df.loc['2018-08-03', 4], 'r') as src:
    band = src.read(1)
    show_band(band, color_map='viridis')

In [ ]:
# Clouds are common problem with remote sensing data. We must get rid of them!
# It is somewhat complicated - you must dig in the Landsat 8 documentation to prepare
# functions for this task. I've prepared it for you as a class PixelQA. We will work with it
# for a while. You may see that methods of this class have many parameters. It is good to know
# them because sometimes your algorithms must be very accurate and sometimes not.

class PixelQA:
    """Class removes clouds and cloud shadows from images based on the quality band given with Landsat's scene.
    Class is initialized with the folder with all Landsat scenes and the QA band.
    
    Class methods:
    prepare values(clear_key='all') - method prepares values to remove with accordance to the Landsat documentation:
    https://landsat.usgs.gov/landsat-surface-reflectance-quality-assessment.
    clear_pixels(band, qa_input = None, clear_by='all', nofill_value=0.0) - method clears pixels of a given band.
    Method takes band for cleaning (band) multiply it with (qa_input) where all values to remove are zeros and
    other values are ones. Removed pixels are filled by nofill_value.
    IMPORTANT:
    Script works only with the Landsat 8 Level-2 data products. To process Level 1 products you should use different
    values for cloud and cloud confidence estimation."""
    
    
    def __init__(self, qa_file=None, data_folder=None):
        
        # prepare qa file address
        if qa_file is None:
            file_list = os.listdir(data_folder)
            for f in file_list:
                if (f.startswith('LC') and f.endswith('pixel_qa.tif')):
                    self.bqa_file = data_folder + f
        else:
            self.bqa_file = qa_file
        
        # prepare qa band
        with rio.open(self.bqa_file) as f:
            self.qa_band = f.read(1)
            
        # Dictionary with cloud pixel values for pixel_qa band 
        # (https://landsat.usgs.gov/landsat-surface-reflectance-quality-assessment)
        self.clouds = {'Cloud Shadow': [328, 392, 840, 904, 1350],
                      'Cloud': [352, 368, 416, 432, 480, 864, 880, 928, 944, 992],
                      'Cloud confidence': {'Low': [322, 324, 328, 336, 352, 368, 834, 836, 840, 848, 864, 880],
                                           'Medium': [386, 388, 392, 400, 416, 432, 900, 904, 928, 944],
                                           'High': [480, 992]},
                      'Cirrus confidence': {'Low': [322, 324, 328, 336, 352, 368, 386, 388, 392, 400, 416, 432, 480],
                                           'High': [834, 836, 840, 848, 864, 880, 898, 900, 904, 912, 928, 944, 992]}}
        
    def prepare_values(self, clear_key):
        """
        Function prepares values to remove (cloud pixels) based on the input.
        input 1: clear_key (string)
        parameters:
        'all' - returns values for Clouds and Cloud Shadows,
        'Cloud Shadow' - returns values for Cloud Shadows,
        'Cloud' - returns values for Clouds,
        'Cloud low' - returns pixel values which could be a cloud with low, medium and high probability,
        'Cloud medium' - returns pixel values which could be a cloud with medium and high probability,
        'Cloud high' - returns pixel values which could be a cloud with high probability,
        'Cirrus low' - returns pixel values which could be a cirrus with low and high probability,
        'Cirrus high' - returns pixel values which could be a cirrus with high probability.
        output: python list with pixel values to remove from the scene"""
        
        values = {'all': [self.clouds['Cloud Shadow'], self.clouds['Cloud']],
                 'Cloud Shadow': [self.clouds['Cloud Shadow']],
                 'Cloud': [self.clouds['Cloud']],
                 'Cloud low': [self.clouds['Cloud confidence']['Low'],
                               self.clouds['Cloud confidence']['Medium'],
                               self.clouds['Cloud confidence']['High']],
                 'Cloud medium': [self.clouds['Cloud confidence']['Medium'],
                                  self.clouds['Cloud confidence']['High']],
                 'Cloud high': [self.clouds['Cloud confidence']['High']],
                 'Cirrus low': [self.clouds['Cirrus confidence']['Low'],
                                self.clouds['Cirrus confidence']['High']],
                 'Cirrus high': [self.clouds['Cirrus confidence']['High']]}
        values_to_remove = values[clear_key]
        
        if len(values_to_remove) > 1:
            vals = []
            for values_list in values_to_remove:
                vals = vals + values_list
        else:
            vals = values_to_remove[0]
        return vals
    
    def clear_pixels(self, band, qa_input = None, clear_by='all', nofill_value=0.0):
        """
        Function removes cloud pixels from the chosen scene based on the scene quality band.
        input 1: band (numpy array) - band for the cloud removal.
        input 2: qa_input - if None then self.qa_band is used as the mask for cloud pixels. Else mask should be
        given as a numpy array.
        input 3: clear_all - parameter to obtain values of cloud pixels:
        'all' - returns values for Clouds and Cloud Shadows,
        'Cloud Shadow' - returns values for Cloud Shadows,
        'Cloud' - returns values for Clouds,
        'Cloud low' - returns pixel values which could be a cloud with low, medium and high probability,
        'Cloud medium' - returns pixel values which could be a cloud with medium and high probability,
        'Cloud high' - returns pixel values which could be a cloud with high probability,
        'Cirrus low' - returns pixel values which could be a cirrus with low and high probability,
        'Cirrus high' - returns pixel values which could be a cirrus with high probability.
        input 4: nofill_value - value which replaces cloud pixels.
        output: numpy array (scene) without cloud pixels."""
        
        values_to_remove = self.prepare_values(clear_by)
        if qa_input is None:
            qa_channel = self.qa_band
        else:
            qa_channel = qa_input
        for val in values_to_remove:
            qa_channel[qa_channel==val] = nofill_value
        qa_channel[qa_channel > 0] = 1
        output_band = band * qa_channel
        return output_band

In [ ]:
# Class gives you opportunity to load QA file initialized only by providing the directory
# with it. Try it - for the same date as for the file with clouds. Then show qa_band instance.

test_qa = PixelQA(data_folder=algal_blooms.landsat_dataframe.loc['2018-08-03', 'DIR'])
show_band(test_qa.qa_band, remove_negative=True)

In [ ]:
# Now you know where clouds are and what is the name of the quality band. It will be good to
# update function clip_all() to handle qa band everytime when you run it. I've added this
# function below with new parameter: qa_band = True. (Parameter is set to True. We imply that
# qa_band is always needed for the analysis).

def clip_all(dataset, clipping_vector, bands=(2, 3, 4), qa_band=True, folder_name='data/clipped/'):
    """Function performs clipping of all Landsat 8 bands given in the DIR column of the df parameter.
    input 1: dataset - dataframe with DIR column where each record points to the directory with a Landsat bands
    input 2: clipping_vector - vector geometry which is used as a area of interest
    input 3: bands=(2, 3, 4) - tuple of bands which should be clipped. Bands 2, 3, 4 are used for the BGR image creation.
    input 4: folder_name='clipped/' - directory where all clipped bands should be stored.
    
    output: [correct, damaged] - list of dicts with keys (dates) and values - correct images
    and damaged images. Damaged images were not process and they not exist in the destination folder."""
    
    data_range = len(dataset)
    damaged = {}
    correct = {}
    for i in range(0, data_range):
        date_str = dataset.index[i]
        images = read_landsat_images(dataset['DIR'][i])
        
        # Cut bands
        for band_number in bands:
            band_to_clip = images[band_number]
            filename = folder_name + 'LC_band' + str(band_number) + '_' + date_str + '.tif'
            try:
                clip_area(clipping_vector, band_to_clip, filename)
            except (rio.errors.WindowError, ValueError):
                print('\nPolygon does not cover raster area. Error occurs when projections are different or raster extent is translated and it does not overlap polygon extent.\n')
                try: 
                    damaged[date_str].append(filename)
                except KeyError:
                    damaged[date_str] = [filename]
                pass
            
            # Check file
            try: 
                damaged[date_str]
                print('File: {} not saved. Raster has not been clipped.'.format(
                filename))
            except KeyError:
                try:
                    with rio.open(filename) as tf:
                         pass
                except rio.RasterioIOError:
                    raise IOError('File was: {} not saved successfully'.format(
                    filename))
                else:
                    print('File: {} saved successfully'.format(
                    filename))
                    try: 
                        correct[date_str].append(filename)
                    except KeyError:
                        correct[date_str] = [filename]
        
        # qa_band
        if qa_band:
            file_list = os.listdir(dataset['DIR'][i])
            for f in file_list:
                if (f.startswith('LC') and f.endswith('pixel_qa.tif')):
                    filename = folder_name + 'LC_band_' + date_str + '_pixel_qa.tif'
                    f = dataset['DIR'][i] + f
                    try:
                        clip_area(clipping_vector, f, filename)
                    except (rio.errors.WindowError, ValueError):
                        print('\nPolygon does not cover raster area. Error occurs when projections are different or raster extent is translated and it does not overlap polygon extent.\n')
                        try: 
                            damaged[date_str].append(filename)
                        except KeyError:
                            damaged[date_str] = [filename]
                        pass
            
                    # Check file
                    try: 
                        damaged[date_str]
                        print('File: {} not saved. Raster has not been clipped.'.format(filename))
                    except KeyError:
                        try:
                            with rio.open(filename) as tf:
                                 pass
                        except rio.RasterioIOError:
                            raise IOError('File was: {} not saved successfully'.format(filename))
                        else:
                            print('QA file: {} saved successfully'.format(filename))
                            try: 
                                correct[date_str].append(filename)
                            except KeyError:
                                correct[date_str] = [filename]
            
    return [correct, damaged]

In [ ]:
# What do we have now? It is a good time for little programming summary, so try below:
# a) Prepare dataframe of Landsat 8 directories with RasterTimeSeries class and its methods
main_directory = 'data/rasters'
l8_dirs = RasterTimeSeries(main_folder=main_directory)
l8_dirs.prepare_directories()
l8_dirs_df = l8_dirs.prepare_dataframe()

# b) Show first five records of a DataFrame created in the previous step. Use pandas' df.head()
l8_dirs_df.head()

In [ ]:
# c) Prepare new vector file (this file is correct for the task, explanation in the article).
# Use correct_vec_address variable and VectorData class

correct_vec_address = 'data/vector/zatoka_v2.shp'
correct_vec = VectorData(correct_vec_address)

# d) Prepare 'all' vector parameters with get_vector_data() method and show it with
# show_vector_data() function

correct_vec.get_vector_data()
correct_vec.show_vector_data()

In [ ]:
# e) Read vector's crs (vec_crs attribute)

print(correct_vec.vec_crs)

# f) And reproject vector geometry to the EPSG: 32634, do not update vector's instance
# Instead create variable with vector geometry

updated_geometry = correct_vec.reproject_geometry(destination_epsg=32634)

In [ ]:
# g) Create geo_dict variable and clip all rasters with qa_band set to True. Use a geometry 
# from the previous point as a mask for clipping. Save files wherever you want, 
# but rather not to the folder with the jupyter notebooks.
# Remember that your:
# - dataset is a DataFrame with Landsat 8 directories,
# - clipping_vector is a geometry from the previous steps
# - bands are (1, 2, 3, 4, 5)
# - qa_band must be written (we want to remove clouds and cloud shadows)
# - destination folder must be there (you have two simple choices: update clip_all() 
# function with os.makedirs method - with checking if folder exists by os.path.exists 
# method or create directory manually)

geo_dict = clip_all(dataset=l8_dirs_df,
        clipping_vector=updated_geometry,
        bands=(1, 2, 3, 4, 5),
        qa_band=True,
        folder_name='data/clipped/')

In [ ]:
# h) Prepare DataFrame with all of these values. Use method:
# Use method pandas.DataFrame.from_dict(dictonary, orient='index', columns = [1, 2, 3, 4, 5, 'qa'])

bands_df = pd.DataFrame.from_dict(geo_dict[0], orient='index', columns=[1, 2, 3, 4, 5, 'qa'])

In [ ]:
# i) Show the first ten records of this dataframe

bands_df.head(10)

In [ ]:
# At this point we have everything to perform analysis. We will test three different indices
# to find the dates when algal bloom exists: Enhanced Vegetation Index, kab1, kab2. I've implemented
# EVI and kab1. Your role is to implement function to calculate kab2.

# Prepare function to calculate EVI index, kab1 index and kab2 index

# EVI is calculated as (bands numbers in square brackets):
# evi = 2.5 * ([B5] - [B4]) / ([B5] + 6 * [B4] - 7.5 * [B2] + 1)
# equation above derived from: 
# https://landsat.usgs.gov/sites/default/files/documents/si_product_guide.pdf
# EVI should be scaled by the factor of 0.00001
# EVI must be in a range between -1 and 1. To avoid problems with further analysis EVI values
# below are set to 0

def calculate_evi(band2, band4, band5):
    with rio.open(band5) as a:
        nir_band = (a.read()[0]/10000).astype(np.float)
    with rio.open(band4) as b:
        r_band = (b.read()[0]/10000).astype(np.float)
    with rio.open(band2) as c:
        b_band = (c.read()[0]/10000).astype(np.float)
        
    numerator = nir_band - r_band
    g = 2.5
    c1 = 6
    c2 = 7.5
    l = 1
    denominator = nir_band + c1 * r_band - c2 * b_band + l
    evi = g * (numerator / denominator)
    evi[evi > 1] = 1
    return evi

# Kab1 is calculated as (bands numbers in square brackets):
# kab1 = 1.67 - 3.94 * ln[B2] + 3.78 * ln[B3]
# You should be prepared for zeros and negative values (you are using logarithm here) so use
# numpy.log1p() insted numpy.log()

def calculate_kab1(band2, band3):
    with rio.open(band2) as a:
        blue_band = (a.read()[0]).astype(np.float)
        blue_band[blue_band < 0] = 0
    with rio.open(band3) as b:
        green_band = (b.read()[0]).astype(np.float)
        green_band[green_band < 0] = 0
        
    log_blue = np.log1p(blue_band)
    log_green = np.log1p(green_band)
    kab1 = 1.67 - 3.94 * log_blue + 3.78 * log_green
    return kab1

# Kab2 is calculated as (bands numbers in square brackets):
# kab2 = 6.92274 - 5.7581 * (ln[B1] / ln[B3])
# You should be prepared for zeros and negative values (you are using logarithm here)

def calculate_kab2(band1, band3):
    with rio.open(band1) as a:
        ultra_blue_band = (a.read()[0]).astype(np.float)
        ultra_blue_band[ultra_blue_band < 0] = 0
    with rio.open(band3) as b:
        green_band = (b.read()[0]).astype(np.float)
        green_band[green_band < 0] = 0
        
    log_blue = np.log1p(ultra_blue_band)
    log_green = np.log1p(green_band)
    kab2 = 6.92274 - 5.7581 * (log_blue / log_green)
    return kab2

# Kab1 and Kab2 are derived from: Kabbara, N., J. Benkhelil, M. Awad, and V. Barale. 2008. 
# Monitoring water quality in the coastal area of Tripoli (Lebanon) using high-resolution 
# satellite data. ISPRS Journal of Photogrammetry and Remote Sensing 63:488–495

In [ ]:
# Now you have all indices. Try to calculate one of it for the date 2015-08-11.
# Remember that your output is a numpy array. You could and should show it. Use 'winter'
# as a color_map.
# To select proper bands use DataFrame.loc[index_name, column_name]

kab2_test = calculate_kab2(bands_df.loc['2015-08-11', 1], bands_df.loc['2015-08-11', 3])
show_band(kab2_test, color_map='winter')

In [ ]:
# It will be good to observe how changes mean of each value with the time. Prepare new columns
# in data frame for the mean of each index. Name columns 'EVI', 'KAB1' and 'KAB2'.
# Below is an example how to perform this task for the 'EVI' column. Your role is to do it
# for 'KAB1' and 'KAB2'.

# EVI
bands_df['EVI'] = bands_df.apply(lambda x: np.nanmean(calculate_evi(x[2], x[4], x[5])), axis=1)

# Kab1
bands_df['KAB1'] = bands_df.apply(lambda x: np.nanmean(calculate_kab1(x[2], x[3])), axis=1)

# Kab2
bands_df['KAB2'] = bands_df.apply(lambda x: np.nanmean(calculate_kab2(x[1], x[3])), axis=1)

In [ ]:
# Print kab1 and kab2 values using matplotlib.pyplot. Plot values as a points not as a lines.
# Use example below (for evi). Plot also line for all values mean (you must calculate this mean).
# The easiest way is to use numpy.mean() function.

# evi
evi_mean = [np.mean(bands_df['EVI']) for x in range(0, len(bands_df['EVI']))]

plt.figure(figsize=(15, 3))
plt.plot(bands_df['EVI'], 'ro')
plt.plot(evi_mean, 'b--')
plt.title('EVI over time')
plt.legend(['evi', 'mean of evi'])
plt.show()

# kab1
kab1_mean = [np.mean(bands_df['KAB1']) for x in range(0, len(bands_df['KAB1']))]

plt.figure(figsize=(15, 3))
plt.plot(bands_df['KAB1'], 'ro')
plt.plot(kab1_mean, 'b--')
plt.title('KAB1 over time')
plt.legend(['kab1', 'mean of kab1'])
plt.show()

# kab2
kab2_mean = [np.mean(bands_df['KAB2']) for x in range(0, len(bands_df['KAB2']))]

plt.figure(figsize=(15, 3))
plt.plot(bands_df['KAB2'], 'ro')
plt.plot(kab2_mean, 'b--')
plt.title('KAB2 over time')
plt.legend(['kab2', 'mean of kab2'])
plt.show()

In [ ]:
# Fast look into a plots shows that Kab1 and Kab2 are very similar indicators.
# In that case we will use only one of them. Which one would you like to use?
# My answer is in the article with explanation why.

# We forget about very important processing step! We should remove clouds from which satellite
# image. Clouds are affecting our calculations because cloud pixels are very bright and
# in the other hand cloud shadows make pixels darker than they are in reality.

# To fix this we will use PixelQA class and before we calculate mean we will delete cloud and
# cloud shadow pixels from a scene. You may use remove_cloud() function with two parameters:
# output band (evi, kab1 or kab2) and quality map for a given index. The example how to use
# it is given for a kab1. Try to do it for evi!

def remove_clouds(path_to_cloud_band, band_to_clean):
    qa_band = PixelQA(path_to_cloud_band)
    cleaned_band = qa_band.clear_pixels(band_to_clean)
    return cleaned_band

# Kab1

test_kab1 = calculate_kab1(bands_df.loc['2013-05-17', 2], bands_df.loc['2013-05-17', 3])
test_kab1 = remove_clouds(bands_df.loc['2013-05-17', 'qa'], test_kab1)
show_band(test_kab1, color_map='winter')

In [ ]:
# evi

test_evi = calculate_evi(bands_df.loc['2013-05-17', 2],
                         bands_df.loc['2013-05-17', 4],
                         bands_df.loc['2013-05-17', 5])
test_evi = remove_clouds(bands_df.loc['2013-05-17', 'qa'], test_evi)
show_band(test_evi, color_map='winter', remove_negative=False)

In [ ]:
# Now you know how to remove clouds and finally you may calculate mean for each index.
# Calculate mean for EVI and kab1 and name new columns as 'EVI-cloudless' and 'KAB1-cloudless'.
# Use examples above as the reference.
# The important point is that we must count mean of values greater than 0 because after
# cloud removing operation we set all unwanted pixels to 0.

def calculate_nanmean_of_positive(idx_func, idx_bands, qa_band):
    retrieved_band = idx_func(*idx_bands)
    cloudless = remove_clouds(qa_band, retrieved_band)
    value = np.nanmean(cloudless[cloudless > 0])
    return value


# EVI-cloudless
bands_df['EVI-cloudless'] = bands_df.apply(
    lambda x: calculate_nanmean_of_positive(calculate_evi,
                                            [x[2], x[4], x[5]],
                                            x['qa']), axis=1)
bands_df['KAB1-cloudless'] = bands_df.apply(
    lambda x: calculate_nanmean_of_positive(calculate_kab1,
                                            [x[2], x[3]],
                                            x['qa']), axis=1)

In [ ]:
# Show EVI and EVI-cloudless on the same plot as points with red and green colors. On the same
# plot draw yellow line represnting mean of EVI and blue line representing mean of EVI-cloudless.
# Show KAB1 and KAB1-cloudless on the same plot as points with red and green colors. On the same
# plot draw yellow line represnting mean of KAB1 and blue line representing mean of KAB1-cloudless.

# evi
evi_mean = [np.mean(bands_df['EVI']) for x in range(0, len(bands_df['EVI']))]
evi_cloudless_mean = [
    np.mean(bands_df['EVI-cloudless']) for x in range(0, len(bands_df['EVI-cloudless']))]

plt.figure(figsize=(15, 3))
plt.plot(bands_df['EVI'], 'ro')
plt.plot(bands_df['EVI-cloudless'], 'go')
plt.plot(evi_mean, 'y--')
plt.plot(evi_cloudless_mean, 'b--')
plt.title('EVI over time')
plt.legend(['evi', 'cloudless evi', 'mean of evi', 'mean of cloudless evi'])
plt.show()

# kab1
kab1_mean = [np.mean(bands_df['KAB1']) for x in range(0, len(bands_df['KAB1']))]
kab1_cloudless_mean = [
    np.mean(bands_df['KAB1-cloudless']) for x in range(0, len(bands_df['KAB1-cloudless']))]

plt.figure(figsize=(15, 3))
plt.plot(bands_df['KAB1'], 'ro')
plt.plot(bands_df['KAB1-cloudless'], 'go')
plt.plot(kab1_mean, 'y--')
plt.plot(kab1_cloudless_mean, 'b--')
plt.title('KAB1 over time')
plt.legend(['kab1', 'cloudless kab1', 'mean of kab1', 'mean of cloudless kab1'])
plt.show()

In [ ]:
# Analysis:
# what is the simplest method of classification of this dataset? What do you think?
# We use simple threshold to find a time when algal blooms have occured.
# But this is a classic example of problem which can be solved which machine learning.
# At the end of this course we will tackle it with a simplest approach and in the future you may
# expect more complicated methods.

# We will set a threshold to the mean of the bands means. The first example is for EVI.
# Try to do the same for the kab1.

# EVI

threshold = np.mean(bands_df['EVI-cloudless'])
algal_blooms_by_evi = bands_df[bands_df['EVI-cloudless'] > threshold]
algal_blooms_by_evi

In [ ]:
# Check... the best option is to look into a EVI image itself

def perform_visual_test_evi(df_of_interest):
    test_band = df_of_interest.apply(lambda x: calculate_evi(x[2],
                                                            x[4],
                                                            x[5]), axis=1)
    indexes = test_band.index.values
    for i in indexes:
        print('Date:', i)
        show_band(test_band[i], color_map='viridis', remove_negative=False)
    

perform_visual_test_evi(algal_blooms_by_evi)

In [ ]:
# And kab1 results

threshold = np.mean(bands_df['KAB1-cloudless'])
algal_blooms_by_kab1 = bands_df[bands_df['KAB1-cloudless'] > threshold]
algal_blooms_by_kab1

In [ ]:
# Checking... the best option is to look into a kab1 image itself

def perform_visual_test_kab1(df_of_interest):
    test_band = df_of_interest.apply(lambda x: calculate_kab1(x[2],
                                                            x[3]), axis=1)
    indexes = test_band.index.values
    for i in indexes:
        print('Date:', i)
        show_band(test_band[i], color_map='viridis', remove_negative=False)
    

perform_visual_test_kab1(algal_blooms_by_kab1)